# History

## Test with and without scaling
- Clean and (vanilla) StandardScaler() min_interval=20 => 0.63142 (less than without scaling/Cleaning) overfit test set?
  - 0.9670995670995671 On the reduce_test_set 
  
- Clean min_interval=20 => 0.63142... Same?  (normal, in random forest it is *useless*)
  - 0.9783549783549783 On the reduce_test_set 

## Params 
- Grid_search "classify__n_estimator": [3, 10, 30], "classify__min_interval": [3, 5, 10]}

## Same n° of classes (yay)

## Not enough tunable parameters in sktime lets do our own model
- Give quite good score (window=16) (less than sktime but still good): 0.9714285714285714 (Better score on Kaggle !!!) - give much more better result by removing nAN (0.73 on kaggle)
- Perform better than a *naive* randomForest given X_train_reduce/Y_train_reduce: 0.9428571428571428
- On the reduce test set having a window=4 give *worst result:* 0.9506493506493506
- Time series model perform better 

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sktime.classification.compose import ColumnEnsembleClassifier
from sktime.classification.dictionary_based import BOSSEnsemble
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.transformations.panel.compose import ColumnConcatenator
from python.utils import load_data
from python.preprocess import TimeSerieMaker, CleanNanByMedian, SimpleSlidingWindowFeaturesExtractor
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from sktime.datatypes._panel._convert import (
    from_3d_numpy_to_nested,
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
DATA_PATH = 'data'
X_train, y_train, X_test = load_data(DATA_PATH)

In [ ]:
X_train_reduce, X_test_reduce, y_train_reduce, y_test_reduce = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

In [ ]:
pipeline_own_rf = Pipeline([
    ("clean", CleanNanByMedian()),
    ("std_scaler", StandardScaler()),
    ("sswfe", SimpleSlidingWindowFeaturesExtractor(window_size=16)),
    ("rnf_cls", RandomForestClassifier(random_state=42))
])

In [ ]:
column_concatenator = [
    ("clean", CleanNanByMedian()),
    ("std_scaler", StandardScaler()),
    ("time-serie", TimeSerieMaker()),
    ("concatenate", ColumnConcatenator()),
    ("classify", TimeSeriesForestClassifier(n_estimators=20, random_state=42, n_jobs=-1, min_interval=3)),
]

time_serie_forest_clf = Pipeline(column_concatenator)

# Split train/test set (leave one out or CV can be interesting) (lance pas ça il faut prepreocess avant et grid search que sur le random forest pas sur la pipeline)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {"n_estimators": [5, 20, 50, 100, 150], "_max_features": [2, 4, 6]}
]
cnm = CleanNanByMedian()
ss = StandardScaler()
SimpleSlidingWindowFeaturesExtractor()
X_train_preprroceessed = SimpleSlidingWindowFeaturesExtractor().fit_transform(StandardScaler().fit_transform(CleanNanByMedian.fit_transform(X_train)))

df_clf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(df_clf, param_grid, cv=10, scoring="accuracy", return_train_score=True)
grid_search.fit(X_train_preprroceessed, y_train)

KeyboardInterrupt: 

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

In [29]:
pipeline_own_rf.fit(X_train, y_train) # best model

Pipeline(steps=[('clean', CleanNanByMedian()), ('std_scaler', StandardScaler()),
                ('sswfe', SimpleSlidingWindowFeaturesExtractor(window_size=16)),
                ('rnf_cls', RandomForestClassifier(random_state=42))])

In [24]:
pipeline_own_rf.score(X_test_reduce, y_test_reduce)

0.9714285714285714

In [17]:
time_serie_forest_clf.fit(X_train_reduce, y_train_reduce) # sktime (perform worst on kaggle)

Pipeline(steps=[('clean', CleanNanByMedian()), ('std_scaler', StandardScaler()),
                ('time-serie', TimeSerieMaker()),
                ('concatenate', ColumnConcatenator()),
                ('classify',
                 TimeSeriesForestClassifier(n_estimators=20, n_jobs=-1,
                                            random_state=42))])

In [18]:
time_serie_forest_clf.score(X_test_reduce, y_test_reduce)

0.9774891774891775

In [30]:
from python.utils import write_submission
y_pred = pipeline_own_rf.predict(X_test)
write_submission(y_pred, 'submissions')

Submission toy_submission.csv saved in submissions/toy_submission.csv.
